In [ ]:
!pip install -q transformers peft accelerate datasets bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.8 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd, os

In [5]:
df = pd.read_csv("/content/problem_summaries.csv")
df = df.rename(columns={"description":"input_text","resume":"target_text"})
df = df.dropna().astype(str)

In [6]:
from datasets import Dataset
ds = Dataset.from_pandas(df[["input_text","target_text"]])
ds = ds.train_test_split(test_size=0.1, seed=42)

In [7]:
base_model = "cointegrated/rut5-small"
tok = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

In [8]:
lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["q","k","v","o","wi_0","wi_1","wo"]  # для T5-подобных
)
model = get_peft_model(model, lora_cfg)

In [9]:
max_src, max_tgt = 256, 64

def preprocess(ex):
    x = tok(ex["input_text"], max_length=max_src, truncation=True)
    y = tok(text_target=ex["target_text"], max_length=max_tgt, truncation=True)
    x["labels"] = y["input_ids"]
    return x

ds_tok = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)
collator = DataCollatorForSeq2Seq(tok, model=model)

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [13]:
!pip install -U transformers

In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="out",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    fp16=True,
    logging_steps=20,
    eval_strategy="steps",   # <-- было evaluation_strategy
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model, args=args,
    data_collator=collator,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["test"]
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=24, training_loss=1.9460954666137695, metrics={'train_runtime': 16.6531, 'train_samples_per_second': 10.809, 'train_steps_per_second': 1.441, 'total_flos': 1112111769600.0, 'train_loss': 1.9460954666137695, 'epoch': 4.0})

In [17]:
ADAPTER_DIR = "zkh_problem_lora"
model.save_pretrained(ADAPTER_DIR)
tok.save_pretrained(ADAPTER_DIR)


Готово. Забери папку: zkh_problem_lora


In [18]:
from google.colab import files
!zip -r zkh_problem_lora.zip zkh_problem_lora
files.download("zkh_problem_lora.zip")


  adding: zkh_problem_lora/ (stored 0%)
  adding: zkh_problem_lora/adapter_config.json (deflated 57%)
  adding: zkh_problem_lora/tokenizer.json (deflated 74%)
  adding: zkh_problem_lora/README.md (deflated 66%)
  adding: zkh_problem_lora/spiece.model (deflated 53%)
  adding: zkh_problem_lora/tokenizer_config.json (deflated 95%)
  adding: zkh_problem_lora/adapter_model.safetensors (deflated 7%)
  adding: zkh_problem_lora/special_tokens_map.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>